In [8]:
caffe_root = '/home/osboxes/caffe/'

In [9]:
import sys

In [10]:
data_root = '/home/osboxes/'

In [14]:
sys.path.insert(0, caffe_root + 'python')

In [15]:
import numpy as np

In [16]:
import caffe

In [17]:
import h5py

In [18]:
import os

In [19]:
import pandas as pd 

In [20]:
import time

In [21]:
if not os.path.isfile(caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'):
    print("Downloading pre-trained CaffeNet model...")
    !caffe_root/scripts/download_model_binary.py ../models/bvlc_reference_caffenet

In [22]:
## Use GPU    
#caffe.set_device(0)
caffe.set_mode_cpu()

In [23]:
def extract_features(images, layer = 'fc7'):
    net = caffe.Net(caffe_root + 'models/bvlc_reference_caffenet/deploy.prototxt',
                caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel',
                caffe.TEST)
    
    # create transformer for the input called 'data'
    transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
    transformer.set_transpose('data', (2,0,1))
    transformer.set_mean('data', np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy').mean(1).mean(1)) # mean pixel
    transformer.set_raw_scale('data', 255)  # rescale from [0,1] to [0,255]
    transformer.set_channel_swap('data', (2,1,0))  # swap channels of RGB to BGR]]

    # copy the image data into the memory allocated for the net
    n_images= len(images)
    net.blobs['data'].reshape(n_images,3,227,227)
    net.blobs['data'].data[...] = map(lambda x: transformer.preprocess('data',caffe.io.load_image(x)), images)
    out = net.forward()

    return net.blobs[layer].data

In [24]:
### Train h5 file

H = h5py.File(data_root+'train_image_C_02features.h5','w')
filenames = H.create_dataset('image_id',(0,), maxshape=(None,),dtype='|S54')
feature = H.create_dataset('features',(0,4096), maxshape = (None,4096))
H.close()

In [25]:
train_pics = pd.read_csv(data_root+'train_photo_to_biz_ids.csv')
train_dir = data_root+'train_photos/'
train_images = [os.path.join(train_dir, str(x)+'.jpg') for x in train_pics['image_id']]
n_train = len(train_images)
print ("Number of training images: ", n_train)


KeyError: 'image_id'

In [ ]:
### Train Images!

### Train h5 File
batch_size = 1000
for i in range(0, n_train, batch_size): 
    images = train_images[i: min(i+batch_size, n_train)]
    features = extract_features(images, layer='fc7')
    n_complete = i+features.shape[0]
    H= h5py.File(data_root+'train_image_C_02features.h5','r+')
    H['image_id'].resize((n_complete,))
    H['image_id'][i: n_complete] = np.array(images)
    H['features'].resize((n_complete,features.shape[1]))
    H['features'][i: n_complete, :] = features
    H.close()
    if n_complete==n_train:
        print ("Step: processing training images completed!!")

In [5]:
### Test h5 file

H = h5py.File(data_root+'test_image_C_02features.h5','w')
filenames = H.create_dataset('image_id',(0,), maxshape=(None,),dtype='|S54')
feature = H.create_dataset('features',(0,4096), maxshape = (None,4096))
H.close()


test_pics = pd.read_csv(data_root+'test_photo_to_biz.csv')
test_dir = data_root+'test_photos/'
test_images = [os.path.join(test_dir, str(x)+'.jpg') for x in test_pics['photo_id'].unique()]  
n_test = len(test_images)


### Test Images!
for i in range(0, num_test, batch_size): 
    images = test_images[i: min(i+batch_size, num_test)]
    features = extract_features(images, layer='fc7')
    num_done = i+features.shape[0]
    
    f= h5py.File(data_root+'test_image_C_02features.h5','r+')
    f['image_id'].resize((num_done,))
    f['image_id'][i: num_done] = np.array(images)
    f['features'].resize((num_done,features.shape[1]))
    f['features'][i: num_done, :] = features
    f.close()
    if num_done==num_test:
        print ("Step: processing test images completed!!")

NameError: name 'data_root' is not defined

In [3]:
### Business Ids -- feature vector -- train

train_photo_to_biz_ids = pd.read_csv(data_root+'train_photo_to_biz_ids.csv')
train_labels = pd.read_csv(data_root+'train.csv').dropna()
train_labels['labels'] = train_labels['labels'].apply(lambda x: tuple(sorted(int(t) for t in x.split())))
train_labels.set_index('business_id', inplace=True)
business_ids = train_labels.index.unique()

### Load image features
H = h5py.File(data_root+'train_image_C_02features.h5','r')
train_image_features = np.copy(H['features'])
H.close()

 
dataframe = pd.DataFrame(columns=['business','label','feature vector'])
index = 0
for business in business_ids:  
    
    label = train_labels.loc[business]['labels']
    image_index = train_photo_to_biz_ids[train_photo_to_biz_ids['business_id']==business].index.tolist()
    train_directory = data_root+'train_photo_folders/'  
    
    features = train_image_features[image_index]
    mean_feature =list(np.mean(features,axis=0))

    dataframe.loc[index] = [business, label, mean_feature]
    index+=1

with open(data_root+"train_biz_C_02features.csv",'w') as H:  
    dataframe.to_csv(H, index=False)

    

NameError: name 'pd' is not defined

In [82]:
### Business Ids -- feature vector  -- test

test_photo_to_biz = pd.read_csv(data_root+'test_photo_to_biz.csv')
business_ids = test_photo_to_biz['business_id'].unique()

### Load image features
H = h5py.File(data_root+'test_image_C_02features.h5','r')
image_filenames = list(np.copy(H['photo_id']))
image_filenames = [name.split('/')[-1][:-4] for name in image_filenames]  #remove the full path and the str ".jpg"
image_features = np.copy(H['features'])
H.close()


dataframe = pd.DataFrame(columns=['business','feature vector'])
index = 0

for business in business_ids:     
    
    image_ids = test_photo_to_biz[test_photo_to_biz['business_id']==business]['photo_id'].tolist()  
    image_index = [image_filenames.index(str(x)) for x in image_ids]
     
    test_directory = data_root+'test_photo_folders/'            
    features = image_features[image_index]
    mean_feature =list(np.mean(features,axis=0))

    dataframe.loc[index] = [business, mean_feature]
    index+=1

with open(data_root+"test_biz_C_02features.csv",'w') as H:  
    dataframe.to_csv(H, index=False)

Number of business:  1996


ValueError: '204149' is not in list

In [ ]:

train_df = pd.read_csv(data_root+"train_biz_C_02_features.csv")
test_df  = pd.read_csv(data_root+"test_biz_C_02_features.csv")

train_label = train_df['label'].values
train_feat = train_df['feature vector'].values
test_feat = test_df['feature vector'].values

def label_to_array(label):
    label = label[1:-1]
    label = label.split(',')
    return [int(l) for l in label if len(l)>0]

def feature_to_vector(feature):
    feature = feature[1:-1]
    feature = feature.split(',')
    return [float(f) for f in str_feature]

train_label = np.array([label_to_array(l) for l in train_df['label']])
train_feat = np.array([label_to_array(f) for f in train_df['feature vector']])
test_feat = np.array([label_to_array(f) for f in test_df['feature vector']])

In [ ]:
from sklearn import svm, datasets
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer

#### Labels to binary matrix

mlb = MultiLabelBinarizer()
b_label= mlb.fit_transform(train_label)  

random_state = np.random.RandomState(0)
train_featpred, test_featpred, b_tlabelpred, b_testlabelpred = train_test_split(train_feat, b_label, test_size=.3,random_state=random_state)
classifier = OneVsRestClassifier(svm.SVC(kernel='linear', probability=True))

# classifier = OneVsRestClassifier(svm.SVC(kernel='rbf', probability=False))

classifier.fit(train_featpred, b_tlabelpred)

predict = classifier.predict(test_featpred)


In [6]:
mlb = MultiLabelBinarizer()
train_label= mlb.fit_transform(train_label)  #Convert list of labels to binary matrix

random_state = np.random.RandomState(0)
classifier = OneVsRestClassifier(svm.SVC(kernel='linear', probability=True))
classifier.fit(train_feat, train_label)

predict2 = classifier.predict(test_feat)

### Binary matrix back to labels
predict2_label = mlb.inverse_transform(predict2) 
  

NameError: name 'MultiLabelBinarizer' is not defined

In [7]:
tdf  = pd.read_csv(data_root+"test_biz_C_02_features.csv")
df = pd.DataFrame(columns=['business_id','labels'])

for i in range(len(tdf)):
    business = tdf.loc[i]['business']
    label = predict_label[i]
    label = str(label)[1:-1].replace(",", " ")
    df.loc[i] = [str(business), label]

with open(data_root+"submission_C_02.csv",'w') as H:
    df.to_csv(H, index=False)  

NameError: name 'data_root' is not defined